## Imports

In [4]:
import tensorflow as tf
import tensorflow.keras as tfk
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle

from utils.hyperparameter import HyperParameter
from utils.tuner import TuningScheduler
from network.layers import HyperModel


from matminer.datasets.convenience_loaders import load_castelli_perovskites
import matminer.featurizers.structure as sfeat

# Alias important packages


#tfkl = tf.keras.layers


# Set seed value
SEED = 10

## Data Processing

### Load dataset

In [54]:
# Load all data from the castelli perovskites dataset
dataframe = load_castelli_perovskites() 
dataframe.head()

,fermi level,fermi width,e_form,gap is direct,structure,mu_b,formula,vbm,cbm,gap gllbsc
0,0.312138,0.001837,2.16,True,"[[0. 0. 0.] Rh, [1.97726555 1.97726555 1.97726...",1.974478e-02,RhTeN3,6.187694,6.187694,0.0
1,0.297083,0.001837,1.52,True,"[[2.54041798 0. 0. ] Hf, [1.020...",-2.253054e-05,HfTeO3,6.033125,6.033125,0.0
2,0.191139,0.003675,1.48,True,"[[0.60790913 0. 0. ] Re, [2.186...",4.982109e+00,ReAsO2F,6.602253,6.602253,0.0
3,0.316346,0.001837,1.24,True,"[[2.83091357 0. 0. ] W, [2.6573...",-8.684496e-01,WReO2S,5.738462,5.738462,0.0
4,0.312658,0.003675,0.62,True,"[[0.00518937 0. 0. ] Bi, [2.172...",2.164069e-15,BiHfO2F,6.074736,6.074736,0.0


In [55]:
# Remove the properties we do not wish to incorporate into our model

dataframe.drop(columns=["formula"]) 

# Isolate material structure

structure_list = dataframe.loc[:,"structure"]

# Define featurizers to convert structural data that is available prior to the scf DFT calculations

DFfeaturizer = sfeat.DensityFeatures("packing fraction")
SCfeaturizer = sfeat.StructuralComplexity(symprec=5e-2) # Featurizer to get the structural complexity of the lattice structures

packing_fraction = []
scomplexity = []

#for id, struct in enumerate(structure_list):
    
Y_data = np.array([dataframe.loc[:,"fermi level"],dataframe.loc[:,"fermi width"],dataframe.loc[:,"e_form"],dataframe.loc[:,"cbm"],dataframe.loc[:,"vbm"]])
X_data = None


## Standard Neural Networks

#### Configure data

In [ ]:
X_shuffled, Y_shuffled = shuffle([X_data,Y_data],random_state=SEED)
X_train,X_test,Y_train,Y_test = train_test_split([X_shuffled,Y_shuffled],test_size=0.1)
X_train,X_val,Y_train,Y_val = train_test_split([X_train,Y_train],test_size=0.2)


batch_size = 250

feature_ndims = X_train.shape[1]

# Here we define the callbacks which we want to use
early_stopping_callback = tfk.callbacks.EarlyStopping(patience=10)

callbacks = [early_stopping_callback]

# The rest of the model is concfigured in the layers.py file - The HyperModel class is constructed in such a way that we sample different hyperparameters

#### Hyperparameter tuning

In [3]:
hyperparameters = HyperParameter()

# Definition of all the hyperparameters that we want sample
hyperparameters.float("learning_rate",min_val=1e-4,max_val=5e-2)
hyperparameters.float("clipvalue",min_val=1e-1,max_val=0.9)
hyperparameters.float("beta_1",min_val=0.87,max_val=0.93)
hyperparameters.float("beta_2",min_val=0.98888,max_val=0.9999)
hyperparameters.float("decay_rate",min_val=0.88,max_val=0.98)
hyperparameters.int("decay_steps",min_val=80,max_val=120)
hyperparameters.boolean("layer_1")
hyperparameters.boolean("layer_2")
hyperparameters.selection("activation_1",selections=["elu","relu"])
hyperparameters.selection("activation_2",selections=["elu","relu"])
hyperparameters.selection("activation_3",selections=["elu","relu"])


hypermodel = HyperModel(feature_ndims)

HPtuner = TuningScheduler(hyperparameters,hypermodel,X_train,Y_train,X_val,Y_val,callbacks)

num_random_configs = 150
num_grid_configs = 10
num_epochs = 1e3

tuned_hparams = HPtuner.rand_tune(num_random_configs,num_epochs,batch_size,verbose=1)

NameError: name 'feature_ndims' is not defined

#### Network Training

#### Model Validation